In [3]:
"""
import requests, json, sys, time, re
from bs4 import BeautifulSoup
from pprint import pprint

#URL = 'https://twitter.com/i/search/timeline?f=tweets&vertical=default&q=google%20OR%20Google%20OR%20%23google%20OR%20%23Google%20OR%20%24GOOG%20OR%20GOOGLE%20OR%20%23GOOGLE%20lang%3Aen%20since%3A2014-01-01%20until%3A2014-01-30&src=typd&include_available_features=1&include_entities=1&lang=en'

#r = requests. get(URL)
#t = r.text

#f = open('scrape_first_page','w')
#f.write(str(t))
#f.close()

with open ("scrape_first_page", "r") as myfile:
    d=myfile.read().replace('\n', '')

data = json. loads(d)

item_html=data['items_html']

pprint(item_html)

print (len(item_html))

html_string = json. dumps(item_html)
soup = BeautifulSoup(html_string, 'html.parser')

for tweet in soup.findAll('p'):
    print (tweet.text)
    print ('\n')
"""
print ("nothing")




nothing


In [122]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
from pprint import pprint

counter = 0
max_tweets = 40
rate_delay = 5


def build_url(since, until, query, max_position=None):
    space = '%20OR%20'
    q = space.join(query)
    s = '-'.join(since)
    u = '-'.join(until)
    s = '%20lang%3Aen%20since%3A'+s
    u = '%20until%3A'+u

    start = 'https://twitter.com/i/search/timeline?f=tweets&vertical=default&q='        
    end   = '&src=typd&include_available_features=1&include_entities=1&lang=en'
    if max_position is not None:
        URL   = start+q+s+u+end+max_position
    else:
        URL   = start+q+s+u+end
    
    return URL
    


def execute_search(url):
    """
    Executes a search to Twitter for the given URL
    :param url: URL to search twitter with
    :return: A JSON object with data from Twitter
    """
    print(url)
    response = requests. get(url)  
    data = json. loads(response.text)
    return data

def parse_tweets(items_html):
    """
    Parses Tweets from the given HTML
    :param items_html: The HTML block with tweets
    :return: A JSON list of tweets
    """
    soup = BeautifulSoup(items_html)
    tweets = []
    for li in soup.find_all("li", class_='js-stream-item'):

        # If our li doesn't have a tweet-id, we skip it as it's not going to be a tweet.
        if 'data-item-id' not in li.attrs:
            continue

        tweet = {
            'tweet_id': li['data-item-id'],
            'text': None,
            'user_id': None,
            'user_screen_name': None,
            'user_name': None,
            'created_at': None,
            'retweets': 0,
            'favorites': 0
        }

        # Tweet Text
        text_p = li.find("p", class_="tweet-text")
        if text_p is not None:
            tweet['text'] = text_p.get_text()

        # Tweet User ID, User Screen Name, User Name
        user_details_div = li.find("div", class_="tweet")
        if user_details_div is not None:
            tweet['user_id'] = user_details_div['data-user-id']
            tweet['user_screen_name'] = user_details_div['data-user-id']
            tweet['user_name'] = user_details_div['data-name']

        # Tweet date
        date_span = li.find("span", class_="_timestamp")
        if date_span is not None:
            tweet['created_at'] = float(date_span['data-time-ms'])

        # Tweet Retweets
        retweet_span = li.select("span.ProfileTweet-action--retweet > span.ProfileTweet-actionCount")
        if retweet_span is not None and len(retweet_span) > 0:
            tweet['retweets'] = int(retweet_span[0]['data-tweet-stat-count'])

        # Tweet Favourites
        favorite_span = li.select("span.ProfileTweet-action--favorite > span.ProfileTweet-actionCount")
        if favorite_span is not None and len(retweet_span) > 0:
            tweet['favorites'] = int(favorite_span[0]['data-tweet-stat-count'])

        tweets.append(tweet)
    return tweets

def save_tweets(tweets):
    """
    Just prints out tweets
    :return:
    """
    global counter
    global max_tweets
    for tweet in tweets:
        # Lets add a counter so we only collect a max number of tweets
        counter += 1

        if tweet['created_at'] is not None:
            t = datetime.datetime.fromtimestamp((tweet['created_at']/1000))
            fmt = "%Y-%m-%d %H:%M:%S"
            #print "%i [%s] - %s" % (counter, t.strftime(fmt), tweet['text'])
            print (str(counter) + ' ' + str(tweet['tweet_id']) + ' [' + str(t.strftime(fmt)) + '] - ' + str(tweet['text']))
            
        # When we've reached our max limit, return False so collection stops
        if counter >= max_tweets:
            return False

    return True
    
def search(since, until, query):
    first = True
    global rate_delay
    url = build_url(since, until, query)
    continue_search = True
    min_tweet = None
    response = execute_search(url)
    while response is not None and continue_search and response['items_html'] is not None:
        tweets = parse_tweets(response['items_html'])
        if first:
            first = False
        else:   
            position = response['min_position']
            pprint(position)

        # If we have no tweets, then we can break the loop early
        if len(tweets) == 0:
            break

        # If we haven't set our min tweet yet, set it now
        if min_tweet is None:
            min_tweet = tweets[0]

        continue_search = save_tweets(tweets)

        # Our max tweet is the last tweet in the list
        max_tweet = tweets[-1]
        if min_tweet['tweet_id'] is not max_tweet['tweet_id']:
            max_position = "&max_position=TWEET-%s-%s-BD1UO2FFu9QAAAAAAAAETAAAAAcAAAASAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA&reset_error_state=false" % (max_tweet['tweet_id'], min_tweet['tweet_id'])
            url = build_url(since, until, query, max_position=max_position)
            #Sleep for our rate_delay
            sleep(rate_delay)
            response = execute_search(url)
    
    
query = ['google', '$GOOG']
since = ['2014', '01', '01']
until = ['2014', '01', '02']
search(since, until, query)

https://twitter.com/i/search/timeline?f=tweets&vertical=default&q=google%20OR%20$GOOG%20lang%3Aen%20since%3A2014-01-01%20until%3A2014-01-02&src=typd&include_available_features=1&include_entities=1&lang=en
1 418532083196514304 [2014-01-01 23:59:59] - Google planning something big with Boston Dynamics - http://www.pingmagazine.tv/news/google-planning-something-big-boston-dynamics/ … pic.twitter.com/mc4AWsdiUE
2 418532075294429184 [2014-01-01 23:59:57] - Google planning something big with Boston Dynamics: Boston Dynamics: Now Belongs to Google… http://goo.gl/fb/1NJTV 
3 418532066767413248 [2014-01-01 23:59:55] - (Ps sorry these aren't mine,me and google are bffs ;3)
4 418532062312689666 [2014-01-01 23:59:54] - If i had to guess, i would guess that the number one search word on Bing is Google.
5 418532059863212032 [2014-01-01 23:59:53] - @troythedecoy @chrisprodromou @muswellhillspur @louise_s_page it goes back years mate, google it.
6 418532054653865984 [2014-01-01 23:59:52] - Wahhahhaa R